In [1]:
import pandas as pd

In [17]:
features_data = pd.read_csv('features.csv')
train_data = pd.read_csv('train.csv')
stores = pd.read_csv('stores.csv')

In [18]:
df = train_data.merge(features_data, on=['Store', 'Date'], how='inner').merge(stores, on=['Store'], how='inner')

In [19]:
df.drop(['IsHoliday_y'], axis=1,inplace=True)
df.rename(columns={'IsHoliday_x':'IsHoliday'},inplace=True)

In [20]:
df[df['IsHoliday']==True]['Date'].unique()

array(['2010-02-12', '2010-09-10', '2010-11-26', '2010-12-31',
       '2011-02-11', '2011-09-09', '2011-11-25', '2011-12-30',
       '2012-02-10', '2012-09-07'], dtype=object)

In [21]:
import numpy as np

In [22]:
df['Is_SuperBowl'] = np.where(df['Date'].isin(['2010-02-12', '2011-02-11', '2012-02-10', '2013-02-08']),1,0)
df['Is_LaborDay'] = np.where(df['Date'].isin(['2010-09-10', '2011-09-09', '2012-09-07', '2013-09-06']),1,0)
df['Is_Thanksgiving'] = np.where(df['Date'].isin(['2010-11-26', '2011-11-25', '2012-11-23', '2013-11-29']),1,0)
df['Is_Christmas'] = np.where(df['Date'].isin(['2010-12-31', '2011-12-30', '2012-12-28', '2013-12-27']),1,0)

In [23]:
df['Weekly_Sales'].describe()

,Weekly_Sales
count,421570.000000
mean,15981.258123
std,22711.183519
min,-4988.940000
25%,2079.650000
50%,7612.030000
75%,20205.852500
max,693099.360000


In [24]:
len(df[df['Weekly_Sales']<0])

1285

In [10]:
df = df[df['Weekly_Sales']>=0]

In [11]:
df.isnull().sum()

,0
Store,0
Dept,0
Date,0
Weekly_Sales,0
IsHoliday,0
Temperature,0
Fuel_Price,0
MarkDown1,270085
MarkDown2,309367
MarkDown3,283618


In [13]:
pip install mlflow dagshub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 73.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 97.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 741.4/741.4 kB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [26]:
import os
import dagshub
import mlflow

os.environ['MLFLOW_TRACKING_USERNAME'] = 'nipkha21'
os.environ['MLFLOW_TRACKING_PASSWORD'] = '202fb8a4c58a90b0eb3598b1037498eb6fe9f593'

dagshub.init(repo_owner='TomC333', repo_name='ml-walmart-recruiting', mlflow=True)

mlflow.set_tracking_uri('https://dagshub.com/TomC333/ml-walmart-recruiting.mlflow')

mlflow.set_experiment("model_experiment_ARIMA")

Initialized MLflow to track repo "TomC333/ml-walmart-recruiting"

Repository TomC333/ml-walmart-recruiting initialized!

<Experiment: artifact_location='mlflow-artifacts:/41361edef7c6479694d4921d6dd5803e', creation_time=1752703107987, experiment_id='4', last_update_time=1752703107987, lifecycle_stage='active', name='Sales_Forecasting_ARIMA_MultiModel', tags={}>

In [15]:
from statsmodels.tsa.arima.model import ARIMA

In [16]:
from sklearn.metrics import mean_absolute_error

In [25]:
df.to_csv('preprocessed_data.csv', index=False)

In [27]:
with mlflow.start_run(run_name="EDA_ARIMA"):
     mlflow.log_param("n_dropped_negative_target_records", 1285)

🏃 View run EDA_ARIMA at: https://dagshub.com/TomC333/ml-walmart-recruiting.mlflow/#/experiments/4/runs/df3d704863804541b70ddecf888d6b17
🧪 View experiment at: https://dagshub.com/TomC333/ml-walmart-recruiting.mlflow/#/experiments/4


In [29]:
with mlflow.start_run(run_name="feature_engineering_ARIMA"):
     mlflow.log_artifact("preprocessed_data.csv", "preprocessed_dataset")
     mlflow.log_param("new_columns", 4)

🏃 View run feature_engineering_ARIMA at: https://dagshub.com/TomC333/ml-walmart-recruiting.mlflow/#/experiments/4/runs/535d54eb528542918660213427a4b0c6
🧪 View experiment at: https://dagshub.com/TomC333/ml-walmart-recruiting.mlflow/#/experiments/4


In [16]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error
import mlflow
import mlflow.statsmodels
import warnings
import pickle
import os
warnings.filterwarnings('ignore')

MIN_DATA_POINTS = 20
TRAIN_RATIO = 0.8

def prepare_store_dept_data(df):
    """Prepare time series data for each store-department combination"""
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values(['Store', 'Dept', 'Date'])

    store_dept_combinations = df.groupby(['Store', 'Dept']).size().reset_index(name='count')

    valid_combinations = store_dept_combinations[
        store_dept_combinations['count'] >= MIN_DATA_POINTS
    ][['Store', 'Dept']]

    print(f"Total store-dept combinations: {len(store_dept_combinations)}")
    print(f"Valid combinations (>= {MIN_DATA_POINTS} data points): {len(valid_combinations)}")

    return df, valid_combinations

def train_arima_for_combination(df, store, dept):
    """Train ARIMA model for specific store-department combination"""
    combo_data = df[(df['Store'] == store) & (df['Dept'] == dept)].copy()
    combo_data = combo_data.sort_values('Date')
    combo_data.set_index('Date', inplace=True)

    combo_data['Weekly_Sales'] = combo_data['Weekly_Sales'].fillna(method='ffill')

    train_size = int(len(combo_data) * TRAIN_RATIO)
    train_data = combo_data['Weekly_Sales'][:train_size]
    test_data = combo_data['Weekly_Sales'][train_size:]

    try:
        model = ARIMA(train_data, order=(1, 1, 1))
        fitted_model = model.fit()

        if len(test_data) > 0:
            forecast = fitted_model.forecast(steps=len(test_data))
            mae = mean_absolute_error(test_data, forecast)
            rmse = np.sqrt(mean_squared_error(test_data, forecast))
            mape = np.mean(np.abs((test_data - forecast) / test_data)) * 100
        else:
            mae = rmse = mape = 0

        return fitted_model, mae, rmse, mape, len(train_data), len(test_data)

    except Exception as e:
        print(f"Error training model for Store {store}, Dept {dept}: {e}")
        return None, None, None, None, None, None


df, valid_combinations = prepare_store_dept_data(df)

models_dict = {}
model_performance = []

mlflow.set_experiment("Sales_Forecasting_ARIMA_MultiModel")

print("Training ARIMA models for each store-department combination...")
print("="*60)

with mlflow.start_run(run_name="model_training_ARIMA"):

    successful_models = 0
    failed_models = 0

    for idx, row in valid_combinations.iterrows():
        store, dept = row['Store'], row['Dept']

        fitted_model, mae, rmse, mape, train_size, test_size = train_arima_for_combination(df, store, dept)

        if fitted_model is not None:
            model_key = f"store_{store}_dept_{dept}"
            models_dict[model_key] = fitted_model

            model_performance.append({
                'Store': store,
                'Dept': dept,
                'MAE': mae,
                'RMSE': rmse,
                'MAPE': mape,
                'Train_Size': train_size,
                'Test_Size': test_size
            })

            successful_models += 1
            print(f"✓ Store {store}, Dept {dept}: MAE={mae:.2f}, RMSE={rmse:.2f}, MAPE={mape:.2f}%")
        else:
            failed_models += 1
            print(f"✗ Failed to train model for Store {store}, Dept {dept}")

    performance_df = pd.DataFrame(model_performance)

    mlflow.log_param("min_data_points", MIN_DATA_POINTS)
    mlflow.log_param("train_ratio", TRAIN_RATIO)
    mlflow.log_param("arima_order", "(1,1,1)")
    mlflow.log_param("total_combinations", len(valid_combinations))
    mlflow.log_param("successful_models", successful_models)
    mlflow.log_param("failed_models", failed_models)

    if len(performance_df) > 0:
        mlflow.log_metric("avg_mae", performance_df['MAE'].mean())
        mlflow.log_metric("avg_rmse", performance_df['RMSE'].mean())
        mlflow.log_metric("avg_mape", performance_df['MAPE'].mean())
        mlflow.log_metric("median_mae", performance_df['MAE'].median())
        mlflow.log_metric("median_rmse", performance_df['RMSE'].median())
        mlflow.log_metric("median_mape", performance_df['MAPE'].median())
        mlflow.log_metric("std_mae", performance_df['MAE'].std())
        mlflow.log_metric("std_rmse", performance_df['RMSE'].std())
        mlflow.log_metric("std_mape", performance_df['MAPE'].std())

    with open('all_models.pkl', 'wb') as f:
        pickle.dump(models_dict, f)


    mlflow.log_artifact('all_models.pkl')
    run_id = mlflow.active_run().info.run_id
    print(run_id)
print("\n" + "="*60)
print("INFERENCE EXAMPLE")
print("="*60)

Total store-dept combinations: 3323
Valid combinations (>= 20 data points): 3071
Training ARIMA models for each store-department combination...
✓ Store 1, Dept 1: MAE=6642.21, RMSE=7474.88, MAPE=37.75%
✓ Store 1, Dept 2: MAE=2733.18, RMSE=3153.00, MAPE=6.07%
✓ Store 1, Dept 3: MAE=7171.96, RMSE=12699.95, MAPE=30.49%
✓ Store 1, Dept 4: MAE=2441.50, RMSE=2863.23, MAPE=6.56%
✓ Store 1, Dept 5: MAE=9170.47, RMSE=9672.36, MAPE=49.77%
✓ Store 1, Dept 6: MAE=1389.01, RMSE=1594.80, MAPE=44.65%
✓ Store 1, Dept 7: MAE=5255.25, RMSE=5977.49, MAPE=28.19%
✓ Store 1, Dept 8: MAE=1841.82, RMSE=2140.23, MAPE=4.88%
✓ Store 1, Dept 9: MAE=5576.59, RMSE=6835.64, MAPE=17.72%
✓ Store 1, Dept 10: MAE=5831.62, RMSE=6442.58, MAPE=19.61%
✓ Store 1, Dept 11: MAE=5541.43, RMSE=6657.79, MAPE=23.47%
✓ Store 1, Dept 12: MAE=930.84, RMSE=1187.53, MAPE=8.76%
✓ Store 1, Dept 13: MAE=2331.24, RMSE=2566.46, MAPE=5.89%
✓ Store 1, Dept 14: MAE=2479.44, RMSE=2816.49, MAPE=19.49%
✓ Store 1, Dept 16: MAE=31799.63, RMSE=34332